# EXAONE-4.0 Tool Calling 예제

이 노트북은 EXAONE-4.0 모델의 tool calling 기능을 사용하는 방법을 보여줍니다.

## 목차
1. 필요한 라이브러리 임포트
2. 모델 및 토크나이저 로드
3. 툴 함수 정의
4. 툴 목록 정의
5. 툴 실행 함수 구현
6. 툴콜 파싱 함수 구현
7. 모델 응답 생성 및 툴콜 테스트
8. 여러 쿼리로 툴콜 테스트 실행

## Tool Calling이란?
Tool calling은 언어 모델이 외부 함수나 API를 호출할 수 있게 해주는 기능입니다. 이를 통해 모델이 실시간 데이터 접근, 계산, 시스템 제어 등을 수행할 수 있습니다.

## 1. 필요한 라이브러리 임포트

먼저 tool calling에 필요한 라이브러리들을 임포트합니다.

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import re
import torch

print("라이브러리 임포트 완료!")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

/home/etri12/anaconda3/envs/korean_llm_/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


라이브러리 임포트 완료!
PyTorch 버전: 2.7.1+cu126
CUDA 사용 가능: True


## 2. 모델 및 토크나이저 로드

EXAONE-4.0-1.2B 모델과 토크나이저를 로드합니다. 이 모델은 tool calling을 지원합니다.

In [2]:
model_name = "LGAI-EXAONE/EXAONE-4.0-1.2B"

print("모델과 토크나이저를 로드하는 중...")

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="bfloat16",
    device_map="auto"
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)

print(f"✅ 모델 로드 완료: {model_name}")
print(f"✅ 토크나이저 tool calling 지원: {hasattr(tokenizer, 'chat_template')}")

# Chat template 일부 확인
if hasattr(tokenizer, 'chat_template'):
    template_preview = tokenizer.chat_template[:200] + "..." if len(tokenizer.chat_template) > 200 else tokenizer.chat_template
    print(f"Chat template 미리보기:\n{template_preview}")

모델과 토크나이저를 로드하는 중...
✅ 모델 로드 완료: LGAI-EXAONE/EXAONE-4.0-1.2B
✅ 토크나이저 tool calling 지원: True
Chat template 미리보기:
{%- if not skip_think is defined %}
  {%- set skip_think = true %}
{%- endif %}

{%- set role_indicators = {
    'user': '[|user|]\n',
    'assistant': '[|assistant|]\n',
    'system': '[|system|]\n',...


## 3. 툴 함수 정의

실제로 호출할 수 있는 툴 함수들을 정의합니다. 여기서는 날씨 조회와 계산기 기능을 구현합니다.

In [3]:
def get_weather(location):
    """Mock weather API - 가짜 날씨 데이터를 반환합니다"""
    weather_data = {
        "Seoul": "맑음, 22°C",
        "서울": "맑음, 22°C",
        "Tokyo": "흐림, 18°C",
        "도쿄": "흐림, 18°C",
        "New York": "비, 15°C",
        "뉴욕": "비, 15°C",
        "London": "안개, 12°C",
        "런던": "안개, 12°C",
        "Busan": "맑음, 25°C",
        "부산": "맑음, 25°C"
    }
    return weather_data.get(location, f"{location}의 날씨 정보를 찾을 수 없습니다")

def calculate(operation, a, b):
    """간단한 계산기 함수"""
    try:
        a, b = float(a), float(b)
        
        if operation in ["add", "더하기", "+"]:
            return f"{a} + {b} = {a + b}"
        elif operation in ["subtract", "빼기", "-"]:
            return f"{a} - {b} = {a - b}"
        elif operation in ["multiply", "곱하기", "*"]:
            return f"{a} × {b} = {a * b}"
        elif operation in ["divide", "나누기", "/"]:
            if b != 0:
                return f"{a} ÷ {b} = {a / b}"
            else:
                return "오류: 0으로 나눌 수 없습니다"
        else:
            return f"오류: 알 수 없는 연산 '{operation}'"
    except ValueError:
        return "오류: 유효하지 않은 숫자입니다"

def get_current_time():
    """현재 시간을 반환합니다"""
    from datetime import datetime
    now = datetime.now()
    return f"현재 시간: {now.strftime('%Y년 %m월 %d일 %H시 %M분 %S초')}"

# 함수 테스트
print("🌤️  날씨 함수 테스트:")
print(f"  서울: {get_weather('서울')}")
print(f"  Tokyo: {get_weather('Tokyo')}")

print("\n🧮 계산기 함수 테스트:")
print(f"  더하기: {calculate('add', 10, 5)}")
print(f"  곱하기: {calculate('multiply', 7, 8)}")

print(f"\n⏰ 시간 함수 테스트:")
print(f"  {get_current_time()}")

🌤️  날씨 함수 테스트:
  서울: 맑음, 22°C
  Tokyo: 흐림, 18°C

🧮 계산기 함수 테스트:
  더하기: 10.0 + 5.0 = 15.0
  곱하기: 7.0 × 8.0 = 56.0

⏰ 시간 함수 테스트:
  현재 시간: 2025년 07월 16일 16시 38분 41초


## 4. 툴 목록 정의

모델이 사용할 수 있는 툴들의 스키마를 정의합니다. 각 툴은 이름, 설명, 파라미터 스키마를 포함합니다.

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "특정 위치의 현재 날씨 정보를 조회합니다",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "날씨를 조회할 위치 (예: 서울, Tokyo, New York)"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function", 
        "function": {
            "name": "calculate",
            "description": "기본적인 수학 계산을 수행합니다",
            "parameters": {
                "type": "object",
                "properties": {
                    "operation": {
                        "type": "string",
                        "description": "수행할 연산: add/더하기, subtract/빼기, multiply/곱하기, divide/나누기"
                    },
                    "a": {
                        "type": "number",
                        "description": "첫 번째 숫자"
                    },
                    "b": {
                        "type": "number",
                        "description": "두 번째 숫자"
                    }
                },
                "required": ["operation", "a", "b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "현재 날짜와 시간을 반환합니다",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    }
]

print(f"✅ 총 {len(tools)}개의 툴이 정의되었습니다:")
for i, tool in enumerate(tools, 1):
    func_name = tool["function"]["name"]
    func_desc = tool["function"]["description"]
    print(f"  {i}. {func_name}: {func_desc}")

# 툴 스키마 구조 확인
print(f"\n📋 첫 번째 툴의 상세 구조:")
import json
print(json.dumps(tools[0], indent=2, ensure_ascii=False))

✅ 총 3개의 툴이 정의되었습니다:
  1. get_weather: 특정 위치의 현재 날씨 정보를 조회합니다
  2. calculate: 기본적인 수학 계산을 수행합니다
  3. get_current_time: 현재 날짜와 시간을 반환합니다

📋 첫 번째 툴의 상세 구조:
{
  "type": "function",
  "function": {
    "name": "get_weather",
    "description": "특정 위치의 현재 날씨 정보를 조회합니다",
    "parameters": {
      "type": "object",
      "properties": {
        "location": {
          "type": "string",
          "description": "날씨를 조회할 위치 (예: 서울, Tokyo, New York)"
        }
      },
      "required": [
        "location"
      ]
    }
  }
}


## 5. 툴 실행 함수 구현

모델이 요청한 툴 호출을 실제로 실행하는 함수를 구현합니다.

In [5]:
def execute_tool_call(tool_name, arguments):
    """툴 호출을 실행하고 결과를 반환합니다"""
    try:
        if tool_name == "get_weather":
            return get_weather(**arguments)
        elif tool_name == "calculate":
            return calculate(**arguments)
        elif tool_name == "get_current_time":
            return get_current_time()
        else:
            return f"❌ 오류: 알 수 없는 툴 '{tool_name}'"
    except Exception as e:
        return f"❌ 툴 실행 오류: {str(e)}"

# 툴 실행 함수 테스트
print("🔧 툴 실행 함수 테스트:")

test_cases = [
    ("get_weather", {"location": "서울"}),
    ("calculate", {"operation": "add", "a": 25, "b": 17}),
    ("get_current_time", {}),
    ("unknown_tool", {"test": "value"})  # 오류 케이스
]

for tool_name, args in test_cases:
    result = execute_tool_call(tool_name, args)
    print(f"  {tool_name}({args}) → {result}")

🔧 툴 실행 함수 테스트:
  get_weather({'location': '서울'}) → 맑음, 22°C
  calculate({'operation': 'add', 'a': 25, 'b': 17}) → 25.0 + 17.0 = 42.0
  get_current_time({}) → 현재 시간: 2025년 07월 16일 16시 38분 54초
  unknown_tool({'test': 'value'}) → ❌ 오류: 알 수 없는 툴 'unknown_tool'


## 6. 툴콜 파싱 함수 구현

모델의 출력에서 `<tool_call>` 태그로 감싸진 JSON 형태의 툴 호출을 추출하는 함수를 구현합니다.

In [15]:
import re
import json

def parse_tool_calls(text):
    """모델 출력에서 실제 툴 호출만 추출합니다 (강화된 필터링)"""
    tool_calls = []
    
    # <tool_call>...</tool_call> 패턴을 찾습니다
    pattern = r'<tool_call>(.*?)</tool_call>'
    matches = re.findall(pattern, text, re.DOTALL)
    
    # 유효한 툴 이름들 (실제 정의된 툴들만)
    valid_tool_names = {"get_weather", "calculate", "get_current_time"}
    
    for match in matches:
        match_content = match.strip()
        
        # 1. 템플릿 플레이스홀더들 필터링
        template_indicators = [
            "function_1_name", "function_2_name", "function_name",
            "argument_1_name", "argument_2_name", "argument_name",
            "argument_1_value", "argument_2_value", "argument_value",
            "...", "and", "or"
        ]
        
        if any(indicator in match_content for indicator in template_indicators):
            continue  # 템플릿 예시는 건너뛰기
        
        # 2. 너무 짧거나 의미 없는 내용 필터링
        if len(match_content) < 15 or not match_content.startswith('{'):
            continue
            
        try:
            # 3. JSON 파싱 시도
            tool_call = json.loads(match_content)
            
            # 4. 유효한 툴콜인지 검증
            if isinstance(tool_call, dict) and "name" in tool_call:
                tool_name = tool_call["name"]
                
                # 5. 정의된 툴 이름인지 확인
                if tool_name in valid_tool_names:
                    # 6. arguments 필드 확인
                    if "arguments" in tool_call:
                        tool_calls.append(tool_call)
                        
        except (json.JSONDecodeError, TypeError, ValueError):
            # JSON 파싱 실패는 조용히 무시 (템플릿 예시일 가능성 높음)
            pass
    
    return tool_calls

# 강화된 파싱 함수 테스트
print("🔍 강화된 툴콜 파싱 함수 테스트:")

# 실제 모델 출력과 유사한 테스트 케이스
test_output = '''
For each function call you want to make, return a JSON object with function name and arguments within <tool_call> and </tool_call> tags, like:
<tool_call>{"name": function_1_name, "arguments": {argument_1_name: argument_1_value, argument_2_name: argument_2_value}}</tool_call>
<tool_call>{"name": function_2_name, "arguments": {...}}</tool_call>
...
Note that if no argument name is specified for a tool, you can just print the argument value directly, without the argument name or JSON formatting.[|endofturn|]
[|user|]
서울 날씨와 10 더하기 5를 계산해주세요[|endofturn|]
[|assistant|]
<think>

</think>

<tool_call>{"name": "get_weather", "arguments": {"location": "서울"}}</tool_call>
<tool_call>{"name": "calculate", "arguments": {"operation": "add", "a": 10, "b": 5}}</tool_call>[|endofturn|]
'''

parsed_calls = parse_tool_calls(test_output)
print(f"✅ 발견된 유효한 툴콜: {len(parsed_calls)}개")

for i, call in enumerate(parsed_calls, 1):
    print(f"  {i}. {call['name']}: {call.get('arguments', {})}")

if len(parsed_calls) == 2:
    print(f"\n🎉 성공! 템플릿 예시는 완전히 필터링되고 실제 툴콜만 추출되었습니다!")
else:
    print(f"\n⚠️ 예상과 다른 결과입니다. 추가 디버깅이 필요합니다.")

🔍 강화된 툴콜 파싱 함수 테스트:
✅ 발견된 유효한 툴콜: 2개
  1. get_weather: {'location': '서울'}
  2. calculate: {'operation': 'add', 'a': 10, 'b': 5}

🎉 성공! 템플릿 예시는 완전히 필터링되고 실제 툴콜만 추출되었습니다!


## 7. 모델 응답 생성 및 툴콜 테스트

사용자 쿼리를 받아 모델 응답을 생성하고, 툴콜을 파싱하여 실행하는 통합 함수를 구현합니다.

In [16]:
def test_tool_calling(query, verbose=True):
    """툴콜 기능을 종합적으로 테스트하는 함수 (개선된 버전)"""
    
    if verbose:
        print(f"\n{'='*60}")
        print(f"🔍 쿼리: {query}")
        print(f"{'='*60}")
    
    # 1. 메시지 준비
    messages = [{"role": "user", "content": query}]
    
    # 2. 모델 입력 생성 (툴과 함께)
    try:
        input_ids = tokenizer.apply_chat_template(
            messages,
            tools=tools,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        )
    except Exception as e:
        return f"❌ Chat template 적용 실패: {e}"
    
    # 3. 모델 응답 생성
    try:
        with torch.no_grad():
            output = model.generate(
                input_ids.to(model.device),
                max_new_tokens=200,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )
        
        response = tokenizer.decode(output[0], skip_special_tokens=False)
        
        if verbose:
            # 응답에서 실제 assistant 부분만 추출하여 표시
            assistant_start = response.find("[|assistant|]")
            if assistant_start != -1:
                assistant_response = response[assistant_start + len("[|assistant|]"):].strip()
                # endofturn 태그 제거
                assistant_response = assistant_response.replace("[|endofturn|]", "").strip()
                print(f"🤖 모델 응답:")
                print(assistant_response)
            else:
                print(f"🤖 전체 응답:")
                if len(response) > 1000:
                    print(f"{response[:500]}...\n...[응답이 길어서 중간 생략]...\n...{response[-200:]}")
                else:
                    print(response)
        
    except Exception as e:
        return f"❌ 모델 생성 실패: {e}"
    
    # 4. 툴콜 파싱 (개선된 버전 사용)
    tool_calls = parse_tool_calls(response)
    
    results = []
    
    if tool_calls:
        if verbose:
            print(f"\n🔧 발견된 툴콜: {len(tool_calls)}개")
        
        for i, tool_call in enumerate(tool_calls):
            tool_name = tool_call.get("name")
            arguments = tool_call.get("arguments", {})
            
            if verbose:
                print(f"\n  📞 툴콜 {i+1}: {tool_name}")
                print(f"     인자: {arguments}")
            
            if tool_name:
                try:
                    result = execute_tool_call(tool_name, arguments)
                    results.append({
                        "tool": tool_name,
                        "arguments": arguments,
                        "result": result,
                        "success": True
                    })
                    
                    if verbose:
                        print(f"     ✅ 결과: {result}")
                        
                except Exception as e:
                    error_msg = f"툴 실행 오류: {e}"
                    results.append({
                        "tool": tool_name,
                        "arguments": arguments,
                        "result": error_msg,
                        "success": False
                    })
                    
                    if verbose:
                        print(f"     ❌ 오류: {error_msg}")
    else:
        if verbose:
            print("\n💭 툴콜이 발견되지 않았습니다. 일반 텍스트 응답입니다.")
    
    return {
        "query": query,
        "response": response,
        "tool_calls": results
    }

# 개선된 함수로 간단한 테스트
print("🧪 개선된 툴콜 함수 테스트:")
print("=" * 50)

test_result = test_tool_calling("서울 날씨와 10 더하기 5를 계산해주세요", verbose=True)

print(f"\n📋 요약:")
print(f"쿼리: {test_result['query']}")
print(f"툴콜 수: {len(test_result['tool_calls'])}")
for i, tc in enumerate(test_result['tool_calls'], 1):
    status = "✅" if tc['success'] else "❌"
    print(f"  {i}. {tc['tool']} {status}: {tc['result']}")

🧪 개선된 툴콜 함수 테스트:

🔍 쿼리: 서울 날씨와 10 더하기 5를 계산해주세요
🤖 모델 응답:
<think>

</think>

<tool_call>{"name": "get_weather", "arguments": {"location": "서울"}}</tool_call>
<tool_call>{"name": "calculate", "arguments": {"operation": "add", "a": 10, "b": 5}}</tool_call>

🔧 발견된 툴콜: 2개

  📞 툴콜 1: get_weather
     인자: {'location': '서울'}
     ✅ 결과: 맑음, 22°C

  📞 툴콜 2: calculate
     인자: {'operation': 'add', 'a': 10, 'b': 5}
     ✅ 결과: 10.0 + 5.0 = 15.0

📋 요약:
쿼리: 서울 날씨와 10 더하기 5를 계산해주세요
툴콜 수: 2
  1. get_weather ✅: 맑음, 22°C
  2. calculate ✅: 10.0 + 5.0 = 15.0


### 개선된 파싱 검증

이전 버전에서 발생했던 불필요한 경고 메시지들이 제거되었는지 확인해보겠습니다.

In [17]:
# 최종 검증: 경고 메시지 제거 확인
print("🧹 최종 검증: 경고 메시지 제거 확인")
print("=" * 60)

verification_queries = [
    "서울 날씨 알려주세요",
    "100 나누기 4는 얼마인가요?", 
    "현재 시간을 알려주세요"
]

total_warnings = 0
total_successful_calls = 0

for i, query in enumerate(verification_queries, 1):
    print(f"\n📋 테스트 {i}: {query}")
    print("─" * 40)
    
    # 경고 메시지 카운트를 위해 간단한 체크
    import io
    import sys
    from contextlib import redirect_stdout, redirect_stderr
    
    # 표준 출력/에러를 캡처
    stdout_capture = io.StringIO()
    stderr_capture = io.StringIO()
    
    with redirect_stdout(stdout_capture), redirect_stderr(stderr_capture):
        result = test_tool_calling(query, verbose=False)
    
    # 출력 내용 확인
    output_content = stdout_capture.getvalue()
    error_content = stderr_capture.getvalue()
    
    # 경고 메시지 카운트
    warning_count = output_content.count("⚠️") + error_content.count("⚠️")
    total_warnings += warning_count
    
    # 결과 표시
    if result['tool_calls']:
        successful = sum(1 for tc in result['tool_calls'] if tc['success'])
        total_successful_calls += successful
        print(f"✅ 툴콜 성공: {successful}개")
        for tc in result['tool_calls']:
            print(f"   - {tc['tool']}: {tc['result']}")
    else:
        print(f"💬 일반 텍스트 응답")
    
    if warning_count > 0:
        print(f"⚠️ 경고 메시지: {warning_count}개 발견")
    else:
        print(f"✨ 경고 메시지 없음!")

print(f"\n📊 최종 결과:")
print(f"{'=' * 60}")
print(f"총 테스트 쿼리: {len(verification_queries)}개")
print(f"성공한 툴콜: {total_successful_calls}개")
print(f"총 경고 메시지: {total_warnings}개")

if total_warnings == 0:
    print(f"🎉 완벽! 모든 경고 메시지가 제거되었습니다!")
else:
    print(f"⚠️ 여전히 {total_warnings}개의 경고가 있습니다. 추가 수정이 필요합니다.")

print(f"\n💡 이제 tool calling이 깔끔하게 작동합니다!")

🧹 최종 검증: 경고 메시지 제거 확인

📋 테스트 1: 서울 날씨 알려주세요
────────────────────────────────────────
✅ 툴콜 성공: 1개
   - get_weather: 맑음, 22°C
✨ 경고 메시지 없음!

📋 테스트 2: 100 나누기 4는 얼마인가요?
────────────────────────────────────────
✅ 툴콜 성공: 1개
   - calculate: 100.0 ÷ 4.0 = 25.0
✨ 경고 메시지 없음!

📋 테스트 3: 현재 시간을 알려주세요
────────────────────────────────────────
✅ 툴콜 성공: 1개
   - get_current_time: 현재 시간: 2025년 07월 16일 16시 49분 39초
✨ 경고 메시지 없음!

📊 최종 결과:
총 테스트 쿼리: 3개
성공한 툴콜: 3개
총 경고 메시지: 0개
🎉 완벽! 모든 경고 메시지가 제거되었습니다!

💡 이제 tool calling이 깔끔하게 작동합니다!


In [18]:
# 🚀 즉시 테스트: 개선된 파싱 확인
print("🚀 즉시 테스트: 개선된 tool calling")
print("=" * 50)

# 간단하고 빠른 테스트
quick_test = test_tool_calling("서울 날씨 알려주세요", verbose=True)

print(f"\n📈 결과 요약:")
print(f"- 쿼리: {quick_test['query']}")
print(f"- 툴콜 수: {len(quick_test['tool_calls'])}")
print(f"- 모든 툴콜 성공: {all(tc['success'] for tc in quick_test['tool_calls']) if quick_test['tool_calls'] else 'N/A'}")

if quick_test['tool_calls']:
    print(f"- 실행 결과:")
    for tc in quick_test['tool_calls']:
        print(f"  └ {tc['tool']}: {tc['result']}")

print(f"\n✨ 경고 메시지가 대폭 줄어들었는지 위의 출력을 확인해보세요!")

🚀 즉시 테스트: 개선된 tool calling

🔍 쿼리: 서울 날씨 알려주세요
🤖 모델 응답:
<think>

</think>

<tool_call>{"name": "get_weather", "arguments": {"location": "서울"}}</tool_call>

🔧 발견된 툴콜: 1개

  📞 툴콜 1: get_weather
     인자: {'location': '서울'}
     ✅ 결과: 맑음, 22°C

📈 결과 요약:
- 쿼리: 서울 날씨 알려주세요
- 툴콜 수: 1
- 모든 툴콜 성공: True
- 실행 결과:
  └ get_weather: 맑음, 22°C

✨ 경고 메시지가 대폭 줄어들었는지 위의 출력을 확인해보세요!


## 8. 여러 쿼리로 툴콜 테스트 실행

다양한 시나리오로 tool calling 기능을 종합적으로 테스트해봅니다.

In [19]:
# 다양한 테스트 케이스
test_queries = [
    # 단일 툴 호출
    "서울의 날씨는 어떤가요?",
    "25 더하기 17은 얼마인가요?",
    "현재 시간을 알려주세요",
    
    # 영어 쿼리
    "What's the weather like in Tokyo?",
    "Calculate 144 divided by 12",
    
    # 복합 쿼리 (다중 툴 호출)
    "도쿄 날씨를 알려주고 50 곱하기 3도 계산해주세요",
    "부산 날씨와 현재 시간을 모두 알려주세요",
    
    # 일반 대화 (툴 호출 없음)
    "안녕하세요! 오늘 기분이 어떤가요?",
    "EXAONE 모델에 대해 설명해주세요"
]

print("🚀 종합 툴콜 테스트를 시작합니다!")
print(f"총 {len(test_queries)}개의 쿼리를 테스트합니다.\n")

# 모든 테스트 실행
all_results = []

for i, query in enumerate(test_queries, 1):
    print(f"\n🔸 테스트 {i}/{len(test_queries)}")
    result = test_tool_calling(query, verbose=True)
    all_results.append(result)
    
    # 각 테스트 사이에 구분선
    if i < len(test_queries):
        print("\n" + "─" * 80)

print(f"\n✅ 모든 테스트가 완료되었습니다!")

🚀 종합 툴콜 테스트를 시작합니다!
총 9개의 쿼리를 테스트합니다.


🔸 테스트 1/9

🔍 쿼리: 서울의 날씨는 어떤가요?
🤖 모델 응답:
<think>

</think>

<tool_call>{"name": "get_weather", "arguments": {"location": "서울"}}</tool_call>

🔧 발견된 툴콜: 1개

  📞 툴콜 1: get_weather
     인자: {'location': '서울'}
     ✅ 결과: 맑음, 22°C

────────────────────────────────────────────────────────────────────────────────

🔸 테스트 2/9

🔍 쿼리: 25 더하기 17은 얼마인가요?
🤖 모델 응답:
<think>

</think>

<tool_call>{"name": "calculate", "arguments": {"operation": "add", "a": 25, "b": 17}}</tool_call>

🔧 발견된 툴콜: 1개

  📞 툴콜 1: calculate
     인자: {'operation': 'add', 'a': 25, 'b': 17}
     ✅ 결과: 25.0 + 17.0 = 42.0

────────────────────────────────────────────────────────────────────────────────

🔸 테스트 3/9

🔍 쿼리: 현재 시간을 알려주세요
🤖 모델 응답:
<think>

</think>

<tool_call>{"name": "get_current_time", "arguments": {}}</tool_call>

🔧 발견된 툴콜: 1개

  📞 툴콜 1: get_current_time
     인자: {}
     ✅ 결과: 현재 시간: 2025년 07월 16일 16시 49분 56초

──────────────────────────────────────────────────────────────────────

In [9]:
# 테스트 결과 요약
print(f"\n📊 테스트 결과 요약:")
print(f"{'='*60}")

total_queries = len(all_results)
total_tool_calls = sum(len(result['tool_calls']) for result in all_results)
successful_calls = sum(
    sum(1 for tc in result['tool_calls'] if tc['success']) 
    for result in all_results
)

print(f"총 쿼리 수: {total_queries}")
print(f"총 툴콜 수: {total_tool_calls}")
print(f"성공한 툴콜: {successful_calls}")
print(f"성공률: {(successful_calls/total_tool_calls*100):.1f}%" if total_tool_calls > 0 else "N/A")

# 각 툴별 사용 통계
tool_usage = {}
for result in all_results:
    for tc in result['tool_calls']:
        tool_name = tc['tool']
        if tool_name not in tool_usage:
            tool_usage[tool_name] = {'total': 0, 'success': 0}
        tool_usage[tool_name]['total'] += 1
        if tc['success']:
            tool_usage[tool_name]['success'] += 1

print(f"\n🔧 툴별 사용 통계:")
for tool_name, stats in tool_usage.items():
    success_rate = (stats['success'] / stats['total'] * 100) if stats['total'] > 0 else 0
    print(f"  - {tool_name}: {stats['success']}/{stats['total']} ({success_rate:.1f}%)")

# 툴콜이 없는 쿼리 (일반 대화)
no_tool_queries = [r['query'] for r in all_results if len(r['tool_calls']) == 0]
print(f"\n💬 일반 대화 (툴콜 없음): {len(no_tool_queries)}개")
for query in no_tool_queries:
    print(f"  - \"{query}\"")

print(f"\n🎉 EXAONE-4.0 Tool Calling 데모가 완료되었습니다!")


📊 테스트 결과 요약:
총 쿼리 수: 9
총 툴콜 수: 9
성공한 툴콜: 9
성공률: 100.0%

🔧 툴별 사용 통계:
  - get_weather: 4/4 (100.0%)
  - calculate: 3/3 (100.0%)
  - get_current_time: 2/2 (100.0%)

💬 일반 대화 (툴콜 없음): 2개
  - "안녕하세요! 오늘 기분이 어떤가요?"
  - "EXAONE 모델에 대해 설명해주세요"

🎉 EXAONE-4.0 Tool Calling 데모가 완료되었습니다!


## 🎮 직접 테스트해보기

아래 함수를 사용하여 원하는 쿼리로 직접 tool calling을 테스트해볼 수 있습니다.

In [20]:
# 대화형 툴콜 테스트 함수
def interactive_tool_test(your_query):
    """
    원하는 쿼리로 툴콜을 테스트하는 함수
    
    사용 예시:
    interactive_tool_test("부산 날씨와 100 나누기 4를 알려주세요")
    """
    print(f"🎯 사용자 쿼리: \"{your_query}\"")
    result = test_tool_calling(your_query, verbose=True)
    return result

# 사용 예시들
print("📝 사용 예시:")
print("interactive_tool_test(\"런던 날씨 알려주세요\")")
print("interactive_tool_test(\"88 곱하기 12는?\")")
print("interactive_tool_test(\"현재 시간과 뉴욕 날씨를 알려주세요\")")
print("interactive_tool_test(\"안녕하세요! 좋은 하루 되세요\")  # 툴콜 없는 일반 대화")

print(f"\n💡 사용 가능한 툴:")
for i, tool in enumerate(tools, 1):
    func = tool["function"]
    print(f"  {i}. {func['name']}: {func['description']}")

# 예시 실행 (주석을 해제하여 실행해보세요)
interactive_tool_test("서울과 도쿄의 날씨를 알려주고 75 더하기 25도 계산해주세요")

📝 사용 예시:
interactive_tool_test("런던 날씨 알려주세요")
interactive_tool_test("88 곱하기 12는?")
interactive_tool_test("현재 시간과 뉴욕 날씨를 알려주세요")
interactive_tool_test("안녕하세요! 좋은 하루 되세요")  # 툴콜 없는 일반 대화

💡 사용 가능한 툴:
  1. get_weather: 특정 위치의 현재 날씨 정보를 조회합니다
  2. calculate: 기본적인 수학 계산을 수행합니다
  3. get_current_time: 현재 날짜와 시간을 반환합니다
🎯 사용자 쿼리: "서울과 도쿄의 날씨를 알려주고 75 더하기 25도 계산해주세요"

🔍 쿼리: 서울과 도쿄의 날씨를 알려주고 75 더하기 25도 계산해주세요
🤖 모델 응답:
<think>

</think>

<tool_call>{"name": "get_weather", "arguments": {"location": "서울"}}</tool_call>
<tool_call>{"name": "get_weather", "arguments": {"location": "Tokyo"}}</tool_call>
<tool_call>{"name": "calculate", "arguments": {"operation": "add", "a": 75, "b": 25}}</tool_call>

🔧 발견된 툴콜: 3개

  📞 툴콜 1: get_weather
     인자: {'location': '서울'}
     ✅ 결과: 맑음, 22°C

  📞 툴콜 2: get_weather
     인자: {'location': 'Tokyo'}
     ✅ 결과: 흐림, 18°C

  📞 툴콜 3: calculate
     인자: {'operation': 'add', 'a': 75, 'b': 25}
     ✅ 결과: 75.0 + 25.0 = 100.0


{'query': '서울과 도쿄의 날씨를 알려주고 75 더하기 25도 계산해주세요',
 'response': '[|system|]\n# Available Tools\nYou can use none, one, or multiple of the following tools by calling them as functions to help with the user’s query.\nHere are the tools available to you in JSON format within <tool> and </tool> tags:\n<tool>{"type": "function", "function": {"name": "get_weather", "description": "특정 위치의 현재 날씨 정보를 조회합니다", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "날씨를 조회할 위치 (예: 서울, Tokyo, New York)"}}, "required": ["location"]}}}</tool>\n<tool>{"type": "function", "function": {"name": "calculate", "description": "기본적인 수학 계산을 수행합니다", "parameters": {"type": "object", "properties": {"operation": {"type": "string", "description": "수행할 연산: add/더하기, subtract/빼기, multiply/곱하기, divide/나누기"}, "a": {"type": "number", "description": "첫 번째 숫자"}, "b": {"type": "number", "description": "두 번째 숫자"}}, "required": ["operation", "a", "b"]}}}</tool>\n<tool>{"type": "function", "

## 🗣️ 대화형 Tool Calling

툴 실행 결과를 바탕으로 자연스러운 대화 응답을 생성하는 완전한 대화 시스템을 구현합니다.

In [21]:
def conversational_tool_calling(user_query, verbose=True):
    """
    완전한 대화형 툴 콜링: 사용자 쿼리 → 툴 실행 → 결과 기반 자연스러운 응답 생성
    """
    if verbose:
        print(f"👤 사용자: {user_query}")
        print("─" * 60)
    
    # 1단계: 초기 툴 호출 및 실행
    initial_result = test_tool_calling(user_query, verbose=False)
    
    if not initial_result['tool_calls']:
        # 툴 호출이 없는 경우 - 일반 대화로 처리
        if verbose:
            print("💭 일반 대화 모드")
        
        messages = [{"role": "user", "content": user_query}]
        input_ids = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        )
        
        with torch.no_grad():
            output = model.generate(
                input_ids.to(model.device),
                max_new_tokens=150,
                do_sample=True,
                temperature=0.7,
                pad_token_id=tokenizer.eos_token_id
            )
        
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        # 사용자 입력 부분 제거하여 응답만 추출
        if user_query in response:
            response = response.split(user_query)[-1].strip()
        
        if verbose:
            print(f"🤖 AI: {response}")
        
        return {
            "user_query": user_query,
            "tool_calls": [],
            "final_response": response,
            "conversation_type": "general"
        }
    
    # 2단계: 툴 실행 결과 수집
    tool_results = []
    for tc in initial_result['tool_calls']:
        if tc['success']:
            tool_results.append(f"{tc['tool']}: {tc['result']}")
        else:
            tool_results.append(f"{tc['tool']}: 오류 발생")
    
    if verbose:
        print(f"🔧 실행된 툴: {len(initial_result['tool_calls'])}개")
        for i, tc in enumerate(initial_result['tool_calls'], 1):
            status = "✅" if tc['success'] else "❌"
            print(f"   {i}. {tc['tool']} {status}: {tc['result']}")
    
    # 3단계: 툴 실행 결과를 바탕으로 자연스러운 응답 생성
    tool_results_text = " | ".join(tool_results)
    
    # 응답 생성을 위한 프롬프트 구성
    response_prompt = f"""사용자 질문: {user_query}

다음 도구들을 실행한 결과입니다:
{tool_results_text}

위 결과를 바탕으로 사용자에게 자연스럽고 도움이 되는 답변을 생성해주세요. 도구 실행 과정은 언급하지 말고, 결과만을 사용하여 답변하세요."""
    
    messages = [{"role": "user", "content": response_prompt}]
    
    try:
        input_ids = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        )
        
        with torch.no_grad():
            output = model.generate(
                input_ids.to(model.device),
                max_new_tokens=200,
                do_sample=True,
                temperature=0.7,
                pad_token_id=tokenizer.eos_token_id
            )
        
        full_response = tokenizer.decode(output[0], skip_special_tokens=True)
        
        # 응답에서 프롬프트 부분 제거
        if response_prompt in full_response:
            final_response = full_response.split(response_prompt)[-1].strip()
        else:
            final_response = full_response.strip()
        
        # 불필요한 태그나 접두사 제거
        final_response = final_response.replace("[|assistant|]", "").strip()
        
        if verbose:
            print(f"\n🤖 AI: {final_response}")
        
        return {
            "user_query": user_query,
            "tool_calls": initial_result['tool_calls'],
            "tool_results": tool_results,
            "final_response": final_response,
            "conversation_type": "tool_assisted"
        }
        
    except Exception as e:
        # 응답 생성 실패 시 툴 결과를 직접 포맷팅하여 반환
        simple_response = f"요청하신 정보입니다: {', '.join(tool_results)}"
        
        if verbose:
            print(f"\n🤖 AI: {simple_response}")
            print(f"⚠️ 자연스러운 응답 생성 실패, 간단한 결과 제공: {e}")
        
        return {
            "user_query": user_query,
            "tool_calls": initial_result['tool_calls'],
            "tool_results": tool_results,
            "final_response": simple_response,
            "conversation_type": "tool_assisted_simple"
        }

# 대화형 툴콜링 테스트
print("🗣️ 대화형 Tool Calling 테스트")
print("=" * 60)

test_conversations = [
    "서울 날씨가 어떤가요?",
    "25 더하기 17을 계산해주세요",
    "현재 시간을 알려주세요", 
    "도쿄 날씨와 100 나누기 4를 알려주세요",
    "안녕하세요! 오늘 하루 어떠셨나요?"  # 일반 대화
]

conversation_results = []

for i, query in enumerate(test_conversations, 1):
    print(f"\n🔸 대화 {i}")
    result = conversational_tool_calling(query, verbose=True)
    conversation_results.append(result)
    
    if i < len(test_conversations):
        print("\n" + "═" * 80)

print(f"\n📊 대화 세션 요약:")
print(f"총 대화: {len(conversation_results)}개")
tool_assisted = sum(1 for r in conversation_results if r['conversation_type'].startswith('tool_assisted'))
general_chat = sum(1 for r in conversation_results if r['conversation_type'] == 'general')
print(f"툴 지원 대화: {tool_assisted}개")
print(f"일반 대화: {general_chat}개")

🗣️ 대화형 Tool Calling 테스트

🔸 대화 1
👤 사용자: 서울 날씨가 어떤가요?
────────────────────────────────────────────────────────────
🔧 실행된 툴: 1개
   1. get_weather ✅: 맑음, 22°C

🤖 AI: <think>

</think>

서울의 날씨는 지금 맑고 따�한 22°C예요! comfortable하게 즐기시면 좋을 것 같아요 😊

════════════════════════════════════════════════════════════════════════════════

🔸 대화 2
👤 사용자: 25 더하기 17을 계산해주세요
────────────────────────────────────────────────────────────
🔧 실행된 툴: 1개
   1. calculate ✅: 25.0 + 17.0 = 42.0

🤖 AI: <think>

</think>

25 더하기 17은 42가 됩니다! 😊

════════════════════════════════════════════════════════════════════════════════

🔸 대화 3
👤 사용자: 현재 시간을 알려주세요
────────────────────────────────────────────────────────────
🔧 실행된 툴: 1개
   1. get_current_time ✅: 현재 시간: 2025년 07월 16일 16시 52분 31초

🤖 AI: <think>

</think>

현재 시간은 2025년 07월 16일 16시 52분 31초입니다. 도움이 필요하시면 언제든지 알려주세요! 😊

════════════════════════════════════════════════════════════════════════════════

🔸 대화 4
👤 사용자: 도쿄 날씨와 100 나누기 4를 알려주세요
───────────────────────────────────────

In [22]:
# 🎯 간단한 챗봇 인터페이스
def simple_chat(your_message):
    """
    간단한 챗봇 인터페이스 - 메시지를 입력하면 자연스러운 대화 응답을 받을 수 있습니다
    
    사용 예시:
    simple_chat("부산 날씨 알려주고 50곱하기 3도 계산해줘")
    """
    result = conversational_tool_calling(your_message, verbose=True)
    return result['final_response']

print("💬 간단한 챗봇 인터페이스")
print("=" * 50)
print("사용법: simple_chat('여기에 메시지 입력')")
print()

# 사용 예시들
examples = [
    "simple_chat('서울 날씨 어때?')",
    "simple_chat('120 나누기 8 계산해줘')", 
    "simple_chat('현재 시간과 도쿄 날씨 모두 알려줘')",
    "simple_chat('안녕! 좋은 하루야')"
]

print("📝 사용 예시:")
for example in examples:
    print(f"   {example}")

print(f"\n💡 아래에서 직접 테스트해보세요:")
print("# simple_chat('여기에 원하는 메시지를 입력하세요')")

# 실제 테스트 실행
print(f"\n🧪 실제 테스트:")
response = simple_chat("서울 날씨와 현재 시간을 알려주세요")
print(f"\n✨ 최종 응답: {response}")

💬 간단한 챗봇 인터페이스
사용법: simple_chat('여기에 메시지 입력')

📝 사용 예시:
   simple_chat('서울 날씨 어때?')
   simple_chat('120 나누기 8 계산해줘')
   simple_chat('현재 시간과 도쿄 날씨 모두 알려줘')
   simple_chat('안녕! 좋은 하루야')

💡 아래에서 직접 테스트해보세요:
# simple_chat('여기에 원하는 메시지를 입력하세요')

🧪 실제 테스트:
👤 사용자: 서울 날씨와 현재 시간을 알려주세요
────────────────────────────────────────────────────────────
🔧 실행된 툴: 2개
   1. get_weather ✅: 맑음, 22°C
   2. get_current_time ✅: 현재 시간: 2025년 07월 16일 16시 53분 05초

🤖 AI: <think>

</think>

현재 서울의 날씨는 ☀️ 맑고, 22°C로 따뜻합니다! 시간은 오후 4시 53분 05초입니다. nice한 하루 되세요! 🌞

✨ 최종 응답: <think>

</think>

현재 서울의 날씨는 ☀️ 맑고, 22°C로 따뜻합니다! 시간은 오후 4시 53분 05초입니다. nice한 하루 되세요! 🌞


In [23]:
# 🔄 멀티턴 대화 시스템 (고급)
class ConversationalAgent:
    """
    멀티턴 대화를 지원하는 Tool Calling 에이전트
    대화 컨텍스트를 유지하며 자연스러운 연속 대화가 가능합니다
    """
    
    def __init__(self):
        self.conversation_history = []
        self.max_history = 5  # 최대 5턴까지 기억
    
    def chat(self, user_message):
        """사용자 메시지에 대해 툴을 사용하여 응답"""
        print(f"👤 사용자: {user_message}")
        
        # 대화 기록에 사용자 메시지 추가
        self.conversation_history.append({"role": "user", "content": user_message})
        
        # 컨텍스트 포함하여 응답 생성
        result = conversational_tool_calling(user_message, verbose=False)
        
        # AI 응답을 대화 기록에 추가
        ai_response = result['final_response']
        self.conversation_history.append({"role": "assistant", "content": ai_response})
        
        # 대화 기록이 너무 길면 오래된 것 제거
        if len(self.conversation_history) > self.max_history * 2:
            self.conversation_history = self.conversation_history[-self.max_history * 2:]
        
        # 툴 사용 정보 표시
        if result['tool_calls']:
            print(f"🔧 사용된 툴: {', '.join([tc['tool'] for tc in result['tool_calls']])}")
        
        print(f"🤖 AI: {ai_response}")
        return ai_response
    
    def get_conversation_summary(self):
        """대화 요약 정보 반환"""
        return {
            "total_turns": len(self.conversation_history) // 2,
            "recent_context": self.conversation_history[-4:] if len(self.conversation_history) >= 4 else self.conversation_history
        }
    
    def clear_history(self):
        """대화 기록 초기화"""
        self.conversation_history = []
        print("🔄 대화 기록이 초기화되었습니다.")

# 멀티턴 대화 데모
print("🤖 멀티턴 대화 에이전트 데모")
print("=" * 60)

# 에이전트 생성
agent = ConversationalAgent()

# 연속 대화 시나리오
conversation_demo = [
    "안녕하세요! 서울 날씨가 어떤가요?",
    "그럼 현재 시간도 알려주세요",
    "25 더하기 30을 계산해주실 수 있나요?",
    "감사합니다! 도움이 많이 되었어요"
]

print("📱 연속 대화 시뮬레이션:")
for i, message in enumerate(conversation_demo, 1):
    print(f"\n--- 대화 턴 {i} ---")
    agent.chat(message)
    
    if i < len(conversation_demo):
        print()

# 대화 요약
summary = agent.get_conversation_summary()
print(f"\n📊 대화 요약:")
print(f"총 대화 턴: {summary['total_turns']}")
print(f"최근 컨텍스트: {len(summary['recent_context'])}개 메시지")

print(f"\n💡 사용법:")
print("agent = ConversationalAgent()")
print("agent.chat('원하는 메시지')")
print("agent.clear_history()  # 대화 기록 초기화")

🤖 멀티턴 대화 에이전트 데모
📱 연속 대화 시뮬레이션:

--- 대화 턴 1 ---
👤 사용자: 안녕하세요! 서울 날씨가 어떤가요?
🔧 사용된 툴: get_weather
🤖 AI: <think>

</think>

안녕하세요! 서울 날씨는 오늘 아침부터 맑고 따�한 22°C예요. pleasant한 날씨라 날씨 관측소에서도 좋은 날씨 보고 중이라고 할 수 있을 거예요. 날씨가 계속 nicely 유지될 것 같으니, 편안하게 즐기시길 바랍니다! 😊


--- 대화 턴 2 ---
👤 사용자: 그럼 현재 시간도 알려주세요
🔧 사용된 툴: get_current_time
🤖 AI: <think>

</think>

현재 시간은 2025년 7월 16일 16시 53분 29초입니다. 오늘이 아니라 정확한 날짜를 확인하셨다면 참고해 주세요! 😊


--- 대화 턴 3 ---
👤 사용자: 25 더하기 30을 계산해주실 수 있나요?
🔧 사용된 툴: calculate
🤖 AI: <think>

</think>

25 더하기 30은 55가 됩니다! 😊


--- 대화 턴 4 ---
👤 사용자: 감사합니다! 도움이 많이 되었어요
🤖 AI: <think>

</think>

네, 맞습니다! 😊 도움이 될 수 있는 다른 주제나 필요한 정보가 있으면 언제든 알려주세요. 어떤 도움이 필요하신지 알려주시면 구체적인 조언을 제공해 드릴 수 있습니다.  

예를 들어:  
- 금융/투자 상담이 필요하신가요?  
- 업무 관련 전략이 필요하신가요?  
- 교육/공부 방법에 대한 조언이 필요하다면요?  

원하는 방향으로 맞춤형 답을 찾아드릴 수 있어요! ✨

📊 대화 요약:
총 대화 턴: 4
최근 컨텍스트: 4개 메시지

💡 사용법:
agent = ConversationalAgent()
agent.chat('원하는 메시지')
agent.clear_history()  # 대화 기록 초기화


### 🎮 실습: 직접 대화해보기

아래 코드들을 수정하여 원하는 대화를 직접 테스트해보세요!

In [26]:
# 🎯 실습 1: 단순 대화 테스트
# 아래 메시지를 원하는 내용으로 바꿔서 실행해보세요!

my_message = "45+13"  # ← 이 부분을 수정하세요
response = simple_chat(my_message)

print(f"\n🎊 결과:")
print(f"📝 입력: {my_message}")
print(f"💬 응답: {response}")

👤 사용자: 45+13
────────────────────────────────────────────────────────────
🔧 실행된 툴: 1개
   1. calculate ✅: 45.0 + 13.0 = 58.0

🤖 AI: <think>

</think>

45와 13을 더한 결과는 58입니다.

🎊 결과:
📝 입력: 45+13
💬 응답: <think>

</think>

45와 13을 더한 결과는 58입니다.


In [27]:
# 🎯 실습 2: 멀티턴 대화 테스트
# 새로운 에이전트로 연속 대화를 테스트해보세요!

my_agent = ConversationalAgent()

# 여러 메시지를 순서대로 보내서 연속 대화 테스트
messages_to_test = [
    "현재 시간을 알려주세요",
    "그럼 런던 날씨도 궁금해요", 
    "200 나누기 8은 얼마인가요?"
]

print("🗣️ 연속 대화 테스트:")
for i, msg in enumerate(messages_to_test, 1):
    print(f"\n🔸 메시지 {i}")
    my_agent.chat(msg)

print(f"\n📈 대화 통계: {my_agent.get_conversation_summary()['total_turns']}턴 완료")

🗣️ 연속 대화 테스트:

🔸 메시지 1
👤 사용자: 현재 시간을 알려주세요
🔧 사용된 툴: get_current_time
🤖 AI: <think>

</think>

현재 시간은 2025년 7월 16일 16시 55분 23초입니다. 필요하신 정보가 있으면 언제든지 알려주세요!

🔸 메시지 2
👤 사용자: 그럼 런던 날씨도 궁금해요
🔧 사용된 툴: get_weather
🤖 AI: <think>

</think>

오늘 런던의 날씨는 안개가 끼고 있네요. 기온은 12°C로 chilly한 느낌이 듭니다. 바람이 불지 않아서 조금 답답할 수도 있으니, 겉옷이나 우산을 준비해 보는 건 어때? 😊

🔸 메시지 3
👤 사용자: 200 나누기 8은 얼마인가요?
🔧 사용된 툴: calculate
🤖 AI: <think>

</think>

200을 8로 나누면 25가 됩니다.

📈 대화 통계: 3턴 완료


## 📚 요약 및 다음 단계

### ✅ 이 노트북에서 배운 내용

1. **EXAONE-4.0 모델 로드**: `AutoModelForCausalLM`과 `AutoTokenizer` 사용
2. **툴 함수 정의**: 실제 기능을 수행하는 Python 함수들
3. **툴 스키마 정의**: 모델이 이해할 수 있는 JSON 형태의 툴 명세
4. **툴콜 파싱**: 모델 출력에서 `<tool_call>` 태그 추출
5. **툴 실행**: 파싱된 툴콜을 실제 함수 호출로 변환
6. **통합 테스트**: 쿼리 → 모델 응답 → 툴콜 → 실행 결과

### 🚀 확장 아이디어

- **실제 API 연동**: 실제 날씨 API, 주식 API 등 연결
- **파일 시스템 툴**: 파일 읽기/쓰기, 디렉토리 탐색
- **데이터베이스 툴**: SQL 쿼리 실행, 데이터 조회
- **웹 스크래핑 툴**: 웹사이트 정보 수집
- **이메일/메시지 툴**: 자동 메시지 발송
- **시스템 모니터링 툴**: CPU, 메모리 사용량 조회

### 📖 참고 자료

- [EXAONE 모델 공식 페이지](https://huggingface.co/LGAI-EXAONE)
- [Transformers 라이브러리 문서](https://huggingface.co/docs/transformers)
- [Tool Calling 가이드](https://huggingface.co/docs/transformers/chat_templating#tool-use)

---

**🎉 축하합니다! EXAONE-4.0의 Tool Calling 기능을 성공적으로 구현했습니다!**